In [1]:
import pandas as pd
import os
import re
import numpy as np

# 자료구조 변수 저장

In [2]:
value_dict = {'총자산':'ifrs-full_Assets', '현금':'ifrs-full_CashAndCashEquivalents',
              '부채':'ifrs-full_Liabilities','지배자산':'ifrs-full_EquityAttributableToOwnersOfParent',
             '매출액':'ifrs-full_Revenue', '매출원가':'ifrs-full_CostOfSales', 
              '매출총이익':'ifrs-full_GrossProfit', '판관비':'dart_TotalSellingGeneralAdministrativeExpenses',
              '영업이익':'dart_OperatingIncomeLoss','계속영업이익':'ifrs-full_ProfitLossFromContinuingOperations',
              '당기순이익':'ifrs-full_ProfitLoss', '지배순이익':'ifrs-full_ProfitLossAttributableToOwnersOfParent',
             '영업활동현금흐름':'ifrs-full_CashFlowsFromUsedInOperatingActivities'}

In [3]:
value_sheet_dict = {'Data_bs': ['총자산','현금','부채','지배자산'],
                    'Data_is':['매출액','매출원가','매출총이익','판관비','영업이익',
                               '당기순이익','지배순이익'],
                    'Data_cis':['매출액','매출원가','매출총이익','판관비','영업이익',
                               '당기순이익','지배순이익'],
                    'Data_cf':['영업활동현금흐름']}

In [4]:
sheet_list = ['Data_bs','Data_is','Data_cis','Data_cf']

# 함수 저장

In [129]:
def data_from_bs (file_name):
    # 재무상태표에서 추출
    sheet_name = sheet_list[0]
    excel_temp = pd.read_excel(file_name, sheet_name=sheet_name,index_col=0)

    # html에서 가져온 excel의 경우 항목명이 다르므로, 여기서 걸러내어 바로 다음 종목으로 넘어가도록 함수 설정
    if excel_temp.iloc[0,0] == 'label_ko':
        raise IndexError
    
    columns = ['stock_code','period']+value_sheet_dict[sheet_name]
    df_temp_total_bs = pd.DataFrame(columns=columns)

    excel_temp = treat_error_in_dartfss(file_name,sheet_name,excel_temp)
    
    # 엑셀 파일에서 숫자로 된 column만 반환 (년도)
    columns_new = []
    p = re.compile('[0-9]+')
    for i in excel_temp.columns:
        m = p.match(i)
        if not m == None:
            columns_new.append(m.group()) #group() : 매치된 문자열 반환

    for value_name in value_sheet_dict[sheet_name]:
        df_temp = excel_temp[excel_temp[excel_temp.columns[0]] == value_dict[value_name]]
        df_temp = df_temp[columns_new].T
        if len(df_temp.columns) != 0:
            df_temp_list = df_temp.iloc[:,0].to_list()
            df_temp_total_bs[value_name]=df_temp_list

    columns_new_end = [column[0:6] for column in columns_new]
    df_temp_total_bs['period']=columns_new_end    
    df_temp_total_bs['stock_code']=file_name[-11:-5]
    
    df_temp_total_bs.sort_values(by='period',ascending=False,inplace=True)
    df_temp_total_bs.drop_duplicates(['period'],keep='first',inplace=True)
    df_temp_total_bs.reset_index(drop=True,inplace=True)
    return df_temp_total_bs

In [6]:
def data_from_is (file_name):
    # 손익계산서에서 추출
    # 기간동안의 값이므로, 년도 단위로 값을 가지게 수정.
    sheet_name = sheet_list[1]
    excel_temp = pd.read_excel(file_name, sheet_name=sheet_name,index_col=0)

    columns = ['stock_code','period']+value_sheet_dict[sheet_name]
    df_temp_total_is = pd.DataFrame(columns=columns)

    excel_temp = treat_error_in_dartfss(file_name,sheet_name,excel_temp)
    
    columns_new = []
    p = re.compile('[0-9]+-[0-9]+')
    for i in excel_temp.columns:
        m = p.match(i)
        if not m == None:
            columns_new.append(m.group()) #group() : 매치된 문자열 반환

    for value_name in value_sheet_dict[sheet_name]:
        df_temp = excel_temp[excel_temp[excel_temp.columns[0]] == value_dict[value_name]]
        df_temp = df_temp[columns_new].T

        df_temp = value_to_year(df_temp)

        df_temp_list = df_temp.iloc[:,0].to_list()
        df_temp_total_is[value_name]=df_temp_list

    columns_new_end = [column[9:15] for column in columns_new]
    df_temp_total_is['period']=columns_new_end
    df_temp_total_is['stock_code']=file_name[-11:-5]
    
    df_temp_total_is.sort_values(by='period',ascending=False,inplace=True)
    df_temp_total_is.drop_duplicates(['period'],keep='first',inplace=True)
    df_temp_total_is.reset_index(drop=True,inplace=True)
    return df_temp_total_is

In [7]:
def data_from_cis (file_name):
    # 손익계산서에서 추출
    # 기간동안의 값이므로, 년도 단위로 값을 가지게 수정.
    sheet_name = sheet_list[2]
    excel_temp = pd.read_excel(file_name, sheet_name=sheet_name,index_col=0)

    columns = ['stock_code','period']+value_sheet_dict[sheet_name]
    df_temp_total_cis = pd.DataFrame(columns=columns)

    excel_temp = treat_error_in_dartfss(file_name,sheet_name,excel_temp)
    
    columns_new = []
    p = re.compile('[0-9]+-[0-9]+')
    for i in excel_temp.columns:
        m = p.match(i)
        if not m == None:
            columns_new.append(m.group()) #group() : 매치된 문자열 반환

    for value_name in value_sheet_dict[sheet_name]:
        df_temp = excel_temp[excel_temp[excel_temp.columns[0]] == value_dict[value_name]]
        df_temp = df_temp[columns_new].T

        df_temp = value_to_year(df_temp)

        df_temp_list = df_temp.iloc[:,0].to_list()
        df_temp_total_cis[value_name]=df_temp_list

    columns_new_end = [column[9:15] for column in columns_new]
    df_temp_total_cis['period']=columns_new_end
    df_temp_total_cis['stock_code']=file_name[-11:-5]
    
    df_temp_total_cis.sort_values(by='period',ascending=False,inplace=True)
    df_temp_total_cis.drop_duplicates(['period'],keep='first',inplace=True)
    df_temp_total_cis.reset_index(drop=True,inplace=True)
    return df_temp_total_cis

In [8]:
def data_from_cf (file_name):
    # 현금흐름표에서 추출
    # 기간동안의 값이므로, 년도 단위로 값을 가지게 수정.
    sheet_name = sheet_list[3]
    excel_temp = pd.read_excel(file_name, sheet_name=sheet_name,index_col=0)

    columns = ['stock_code','period']+value_sheet_dict[sheet_name]
    df_temp_total_cf = pd.DataFrame(columns=columns)

    excel_temp = treat_error_in_dartfss(file_name,sheet_name,excel_temp)
    
    columns_new = []
    p = re.compile('[0-9]+-[0-9]+')
    for i in excel_temp.columns:
        m = p.match(i)
        if not m == None:
            columns_new.append(m.group()) #group() : 매치된 문자열 반환

    for value_name in value_sheet_dict[sheet_name]:
        df_temp = excel_temp[excel_temp[excel_temp.columns[0]] == value_dict[value_name]]
        df_temp = df_temp[columns_new].T

        df_temp = value_to_year(df_temp)

        df_temp_list = df_temp.iloc[:,0].to_list()
        df_temp_total_cf[value_name]=df_temp_list

    columns_new_end = [column[9:15] for column in columns_new]
    df_temp_total_cf['period']=columns_new_end
    df_temp_total_cf['stock_code']=file_name[-11:-5]
    
    df_temp_total_cf.sort_values(by='period',ascending=False,inplace=True)
    df_temp_total_cf.drop_duplicates(['period'],keep='first',inplace=True)
    df_temp_total_cf.reset_index(drop=True,inplace=True)
    return df_temp_total_cf

In [9]:
def value_to_year (value_df):
    
    period_df = period_to_year(value_df.index)
#     print(period_df)
    
    for i in range(len(value_df.index)):
        period = value_df.index[i]
#         print(period_df[period_df.period == period])
        add1, add2, del1 = period_df[period_df.period == period].iloc[0,3:6]
        try:
            value0 = value_df.loc[period,value_df.columns[0]]

            if np.isnan(add1):
                value1 = 0
            else:
                value1 = value_df.iloc[int(add1), 0]

            if np.isnan(add2):
                value2 = 0
            else:
                value2 = value_df.iloc[int(add2), 0]

            if np.isnan(del1):
                value3 = 0
            else:
                value3 = value_df.iloc[int(del1), 0]

            value = value0 + value1 + value2 - value3
        except:
            value = np.NaN

        value_df.loc[period,'value'] = value

    value_df.index = period_df['end']
    value_df = value_df.loc[:,['value']]
    
    return value_df

In [10]:
def find_index(data, target):
    res = []
    lis = data
    while True:
        try:
            res.append(lis.index(target) + (res[-1]+1 if len(res)!=0 else 0))
            lis = data[res[-1]+1:]
        except:
            break     
    return res

In [11]:
def period_to_year (periods):
    # 기간을 시작지점과 끝지점으로 분리
    periods_start = [period.split('-')[0][:6] for period in periods]
    periods_end = [period.split('-')[1][:6] for period in periods]
    
    add_list1 = []
    add_list2 = []
    del_list1 = []
    
    for i in range(len(periods)):
        period_start = periods_start[i]
        period_end = periods_end[i]

        if period_start[4:] == '01':
            add_list1.append(np.NaN)
        else:
            try:
                end_target_index = find_index(periods_end, str(int(period_start)-1)) #이번 기간의 전 기간에서 끝나는 index
                start_target_index = find_index(periods_start, period_start[:4]+'01') #해당 연도의 1월부터 시작하는 index
                intersection = list(set(end_target_index) & set(start_target_index)) #교집합을 더하면 1월부터 col_end까지 계산가능
                add_list1.append(int(intersection[0]))
            except:
                add_list1.append('Error')

        if period_end[4:] == '12':
            add_list2.append(np.NaN)
            del_list1.append(np.NaN)
        else:
            try:
                end_target_index_add = find_index (periods_end, str(int(period_end[:4])-1)+'12' )
                start_target_index = find_index(periods_start, str(int(period_end[:4])-1)+'01' )
                intersection_add = list(set(end_target_index_add) & set(start_target_index))
                add_list2.append(int(intersection_add[0]))      
            except:
                add_list2.append('Error')

            try:
                end_target_index_del = find_index (periods_end, str(int(period_end[:4])-1)+period_end[4:] )
                start_target_index = find_index(periods_start, str(int(period_end[:4])-1)+'01' )
                intersection_del = list(set(end_target_index_del) & set(start_target_index))
                del_list1.append(int(intersection_del[0]))
            except:
                del_list1.append('Error')
    
#     print(len(add_list1), len(add_list2), len(del_list1))
    dic = {'period':periods,'start':periods_start,'end':periods_end,'add1':add_list1,'add2':add_list2,'del':del_list1}
    
    return pd.DataFrame(dic)

# 자료 가져오기

In [116]:
data_path_kospi = './Raw/KOSPI by dartfss/'
data_path_kosdaq = './Raw/KOSDAQ by dartfss/'
data_path_del = './Raw/DEL by dartfss/'
files_list_kospi = os.listdir(data_path_kospi)
files_list_kosdaq = os.listdir(data_path_kosdaq)
files_list_del = os.listdir(data_path_del)

In [ ]:
i=0
file_name = data_path_kospi+files_list_kospi[i]

In [142]:
def treat_error_in_dartfss (file_name, sheet_name, excel_temp):
    if (file_name[-11:-5], sheet_name) == ('028260','Data_cis'):
        excel_temp.drop(columns = ['20140401-20140630-20140101-20140630','20140701-20140930-20140101-20140930'], axis=1,inplace=True)
        excel_temp.rename(columns = {'20130401-20130630-20130101-20130630':'20130101-20130630',
                                     '20120401-20120630-20120101-20120630':'20120101-20120630',
                                     '20130701-20130930-20130101-20130930':'20130101-20130930',
                                     '20120701-20120930-20120101-20120930':'20120101-20120930'},inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('034830','Data_bs'):
        raise IndexError
    elif (file_name[-11:-5], sheet_name) == ('064350','Data_cis'):
        excel_temp.drop(columns = ['20130401-20130630-20130101-20130630'], axis=1,inplace=True)
        excel_temp.rename(columns = {'20120401-20120630-20120101-20120630':'20120101-20120630'},inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('033430','Data_bs'):
        raise IndexError
    elif (file_name[-11:-5], sheet_name) == ('053580','Data_bs'):
        excel_temp.drop(columns = ['19000101','20130101-20131231','20120101-20121231'], axis=1,inplace=True)
        excel_temp.rename(columns = {'20140101-20140630':'20140630',
                                     '20140101-20140331':'20140331'},inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('053580','Data_ci'):
        raise IndexError
    elif (file_name[-11:-5], sheet_name) == ('078020','Data_cis'):
        excel_temp.drop(columns = ['19000101','20130401-20130930-20130401-20131231-20120401-20130331',
                                   '20130401-20131231-20120401-20130331',
                                   '20130401-20130630-20130401-20131231-20120401-20130331'], axis=1,inplace=True)
        excel_temp.rename(columns = {'20200401-20200630-20200101-20200630':'20200101-20200630',
                                     '20190401-20190630-20190101-20190630':'20190101-20190630',
                                     '20180401-20180630-20180101-20180630':'20180101-20180630',
                                     '20200701-20200930-20200101-20200930':'20200101-20200930',
                                     '20190701-20190930-20190101-20190930':'20190101-20190930',
                                     '20180701-20180930-20180101-20180930':'20180101-20180930'},inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('101670','Data_ci'):
        raise IndexError
    elif (file_name[-11:-5], sheet_name) == ('101670','Data_cis'):
        excel_temp.drop(columns = ['20130401-20130630-20130101-20130630','20130301-20130630',
                                   '20120401-20120630-20120101-20120630','20120301-20120630',], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('198440','Data_bs'):
        excel_temp.drop(columns = ['20161231-20160220','19000101','20151231-20150120','20141231',
                                   '20150331-20150120'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('198440','Data_cis'):
        excel_temp.drop(columns = ['20160101-20161231-20150101-20151231','19000101','20150101-20151231-20140428-20141231',
                                   '20140428-20141231','20160101-20160331','20150101-20150331-20140428-20141231'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('198440','Data_cf'):
        excel_temp.drop(columns = ['20160101-20161231-20150101-20151231','19000101','20150101-20151231-20140428-20141231',
                                   '20140428-20141231','20160101-20160331','20150101-20150331-20140428-20141231'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('204840','Data_bs'):
        excel_temp.drop(columns = ['20140729-20141231'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('217500','Data_bs'):
        excel_temp.drop(columns = ['20160930-20160120','19000101','20160331-20160120'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('217600','Data_bs'):
        excel_temp.drop(columns = ['20160930-20160120','19000101','20160331-20160120'], axis=1,inplace=True)
    elif (file_name[-11:-5], sheet_name) == ('220630','Data_bs'):
        excel_temp.drop(columns = ['20160331-20160120','19000101','20150930'], axis=1,inplace=True)        
    elif (file_name[-11:-5], sheet_name) == ('307180','Data_bs'):
        excel_temp.drop(columns = ['20190101-20190331','20180831-20181231'], axis=1,inplace=True)  
# DEL LIST
    elif (file_name[-11:-5], sheet_name) == ('103160','Data_cis'):
        excel_temp.drop(columns = ['20120101-20121231-20110101-20111231','19000101',
                                  '20140101-20140331-20130101-20130331','20130101-20130331-20120101-20120331'], axis=1,inplace=True)  
    elif (file_name[-11:-5], sheet_name) == ('103160','Data_cf'):
        excel_temp.drop(columns = ['20120101-20121231-20110101-20111231','19000101',
                                  '20140101-20140331-20130101-20130331','20130101-20130331-20120101-20120331'], axis=1,inplace=True)  
        
        
    return excel_temp

In [128]:
file_name = data_path_kosdaq + '019210.xlsx'

sheet_name = sheet_list[0]
excel_temp = pd.read_excel(file_name, sheet_name=sheet_name,index_col=0)

if excel_temp.iloc[0,0] == 'label_ko':
    raise IndexError

columns = ['stock_code','period']+value_sheet_dict[sheet_name]
df_temp_total_bs = pd.DataFrame(columns=columns)

excel_temp = treat_error_in_dartfss(file_name,sheet_name,excel_temp)

# 엑셀 파일에서 숫자로 된 column만 반환 (년도)
columns_new = []
p = re.compile('[0-9]+')
for i in excel_temp.columns:
    m = p.match(i)
    if not m == None:
        columns_new.append(m.group()) #group() : 매치된 문자열 반환

for value_name in value_sheet_dict[sheet_name]:
    df_temp = excel_temp[excel_temp[excel_temp.columns[0]] == value_dict[value_name]]
    df_temp = df_temp[columns_new].T
    if len(df_temp.columns) != 0:
        df_temp_list = df_temp.iloc[:,0].to_list()
        df_temp_total_bs[value_name]=df_temp_list

columns_new_end = [column[0:6] for column in columns_new]
df_temp_total_bs['period']=columns_new_end    
df_temp_total_bs['stock_code']=file_name[-11:-5]

df_temp_total_bs.sort_values(by='period',ascending=False,inplace=True)
df_temp_total_bs.drop_duplicates(['period'],keep='first',inplace=True)
df_temp_total_bs.reset_index(drop=True,inplace=True)

IndexError: 

In [144]:
# 연결 & 별도가 섞여서 발생하는 문제가 없나?
df_total = pd.DataFrame()
for i in range(len(files_list_del)):
    file_name = data_path_del+files_list_del[i]
    print(file_name)
    try:
        df_temp_total_bs = data_from_bs(file_name)
    except IndexError:
        continue # html로 가져와서 ifrs 자료형이 없는 경우 다음 종목오로 이동.
    try:
        df_temp_total_is = data_from_is(file_name)
    except:
        df_temp_total_is = data_from_cis(file_name)
    df_temp_total_cf = data_from_cf(file_name)
    
    df_temp_total = pd.merge(df_temp_total_bs, df_temp_total_is,on=['stock_code','period'],how='outer')
    df_temp_total = pd.merge(df_temp_total, df_temp_total_cf, on=['stock_code','period'],how='outer')
    
    df_temp_total.drop(axis=0, index = df_temp_total[df_temp_total.T.isnull().sum() == 12].index,inplace=True)
    
    df_total = pd.concat([df_total,df_temp_total],axis=0)

./Raw/DEL by dartfss/000010.xlsx
./Raw/DEL by dartfss/000030.xlsx
./Raw/DEL by dartfss/000110.xlsx
./Raw/DEL by dartfss/000360.xlsx
./Raw/DEL by dartfss/000420.xlsx
./Raw/DEL by dartfss/000470.xlsx
./Raw/DEL by dartfss/000800.xlsx
./Raw/DEL by dartfss/000830.xlsx
./Raw/DEL by dartfss/001300.xlsx
./Raw/DEL by dartfss/001310.xlsx
./Raw/DEL by dartfss/001490.xlsx
./Raw/DEL by dartfss/001590.xlsx
./Raw/DEL by dartfss/001890.xlsx
./Raw/DEL by dartfss/001970.xlsx
./Raw/DEL by dartfss/002000.xlsx
./Raw/DEL by dartfss/002250.xlsx
./Raw/DEL by dartfss/002300.xlsx
./Raw/DEL by dartfss/002480.xlsx
./Raw/DEL by dartfss/002530.xlsx
./Raw/DEL by dartfss/002550.xlsx
./Raw/DEL by dartfss/002670.xlsx
./Raw/DEL by dartfss/002860.xlsx
./Raw/DEL by dartfss/003050.xlsx
./Raw/DEL by dartfss/003190.xlsx
./Raw/DEL by dartfss/003450.xlsx
./Raw/DEL by dartfss/003600.xlsx
./Raw/DEL by dartfss/003640.xlsx
./Raw/DEL by dartfss/003940.xlsx
./Raw/DEL by dartfss/004010.xlsx
./Raw/DEL by dartfss/004130.xlsx
./Raw/DEL 

./Raw/DEL by dartfss/081940.xlsx
./Raw/DEL by dartfss/081970.xlsx
./Raw/DEL by dartfss/082240.xlsx
./Raw/DEL by dartfss/082250.xlsx
./Raw/DEL by dartfss/082260.xlsx
./Raw/DEL by dartfss/082390.xlsx
./Raw/DEL by dartfss/082930.xlsx
./Raw/DEL by dartfss/083350.xlsx
./Raw/DEL by dartfss/083360.xlsx
./Raw/DEL by dartfss/083370.xlsx
./Raw/DEL by dartfss/083380.xlsx
./Raw/DEL by dartfss/083390.xlsx
./Raw/DEL by dartfss/083570.xlsx
./Raw/DEL by dartfss/083580.xlsx
./Raw/DEL by dartfss/083590.xlsx
./Raw/DEL by dartfss/083600.xlsx
./Raw/DEL by dartfss/083610.xlsx
./Raw/DEL by dartfss/083620.xlsx
./Raw/DEL by dartfss/084450.xlsx
./Raw/DEL by dartfss/085990.xlsx
./Raw/DEL by dartfss/086200.xlsx
./Raw/DEL by dartfss/086830.xlsx
./Raw/DEL by dartfss/087220.xlsx
./Raw/DEL by dartfss/088700.xlsx
./Raw/DEL by dartfss/089480.xlsx
./Raw/DEL by dartfss/090120.xlsx
./Raw/DEL by dartfss/090540.xlsx
./Raw/DEL by dartfss/090730.xlsx
./Raw/DEL by dartfss/090970.xlsx
./Raw/DEL by dartfss/090980.xlsx
./Raw/DEL 

In [140]:
files_list_del.index('103160.xlsx')

296

In [145]:
len(set(df_total.stock_code))

177

In [146]:
df_total.to_csv('./Raw_Finance/del_finance.csv',encoding='utf-8-sig')

In [126]:
len(files_list_kospi)

799

In [125]:
set(df_total['stock_code'])

{'000020', '000040', '000050', '000070'}